In [1]:
from azureml.core import Workspace, Experiment
import pandas as pd



ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code ACVXV4YVR to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-130244
Azure region: southcentralus
Subscription id: b3fb99ab-ce93-4e7d-92ea-a05123349c1b
Resource group: aml-quickstarts-130244


In [2]:
!pip install --upgrade --force-reinstall -r https://aka.ms/automl_linux_requirements.txt

     |████████████████████████████████| 55 kB 1.3 MB/s eta 0:00:011
     |████████████████████████████████| 58 kB 2.3 MB/s  eta 0:00:01
     |████████████████████████████████| 124 kB 8.2 MB/s eta 0:00:01
     |████████████████████████████████| 141 kB 9.0 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 2.4 MB/s  eta 0:00:01
     |████████████████████████████████| 94 kB 923 kB/s  eta 0:00:01
     |████████████████████████████████| 718 kB 13.5 MB/s eta 0:00:01
     |████████████████████████████████| 89 kB 4.2 MB/s  eta 0:00:01
     |████████████████████████████████| 1.0 MB 31.1 MB/s eta 0:00:01
     |████████████████████████████████| 547 kB 42.3 MB/s eta 0:00:01
     |████████████████████████████████| 131 kB 47.8 MB/s eta 0:00:01
     |████████████████████████████████| 188 kB 65.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 59.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 24.1 MB/s eta 0:00:01
     |████████████████████████████████| 1

In [3]:
!pip install --upgrade --force-reinstall -r https://automlcesdkdataresources.blob.core.windows.net/validated-requirements/1.19.0/validated_linux_requirements.txt

  Using cached adal-1.2.5-py2.py3-none-any.whl (55 kB)
  Using cached applicationinsights-0.11.9-py2.py3-none-any.whl (58 kB)
  Using cached azure_common-1.1.26-py2.py3-none-any.whl (12 kB)
  Using cached azure_core-1.9.0-py2.py3-none-any.whl (124 kB)
  Using cached azure_graphrbac-0.61.1-py2.py3-none-any.whl (141 kB)
  Using cached azure_identity-1.4.1-py2.py3-none-any.whl (86 kB)
  Using cached azure_mgmt_authorization-0.61.0-py2.py3-none-any.whl (94 kB)
  Using cached azure_mgmt_containerregistry-2.8.0-py2.py3-none-any.whl (718 kB)
  Using cached azure_mgmt_keyvault-2.2.0-py2.py3-none-any.whl (89 kB)
  Using cached azure_mgmt_resource-10.3.0-py2.py3-none-any.whl (1.0 MB)
  Using cached azure_mgmt_storage-11.2.0-py2.py3-none-any.whl (547 kB)
  Using cached azure_storage_queue-12.1.4-py2.py3-none-any.whl (131 kB)
  Using cached azureml_automl_core-1.19.0-py3-none-any.whl (188 kB)
  Using cached azureml_automl_runtime-1.19.0-py3-none-any.whl (1.9 MB)
  Using cached azureml_core-1.19.0-

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster

cpu_cluster_name = 'cpucluster'

# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)


### YOUR CODE HERE ###

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({"--C" : uniform(0.1,1),
                            "--max_iter": choice(50,100,150,200)})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1,evaluation_interval = 1,delay_evaluation=5)
### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = "./training",entry_script="train.py",compute_target=cpu_cluster_name)
### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config =  HyperDriveConfig(
    hyperparameter_sampling=ps,
    primary_metric_name='Accuracy',
    max_total_runs=100,
    max_concurrent_runs=4,
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    policy=policy,
    estimator=est
)

### YOUR CODE HERE ###

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [21]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

experiment=Experiment(ws,ws.name)
hiperdrive_run = experiment.submit(hyperdrive_config)

RunDetails(hiperdrive_run).show()
### YOUR CODE HERE ###

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [7]:
import joblib
# Get your best run and save the model from that run.
best_run = hiperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Regularization Strength: ',parameter_values[1])
print('\n Max iterations: ',parameter_values[3])



### YOUR CODE HERE ###

AttributeError: 'NoneType' object has no attribute 'get_metrics'

In [23]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
from azureml.core import Workspace, Dataset



dataset = Dataset.get_by_name(ws, name='Bank-marketing')
ds = dataset.to_pandas_dataframe()

### YOUR CODE HERE ###

In [24]:
from training.train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

df = x.join(y)

In [25]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=df,
    label_column_name='y',
    n_cross_validations=5)

In [26]:
# Submit your automl run

# experiment=Experiment(ws,ws.name)

local_run = experiment.submit(automl_config, show_output = True)
### YOUR CODE HERE ###

ValidationException: ValidationException:
	Message: Install the required versions of packages using the requirements file. Requirements file location: /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/validated_linux_requirements.txt. Alternatively, use remote target to avoid dependency management. 
Package name/Required version/Installed version
azureml-core/pyopenssl<20.0.0/pyOpenSSL 20.0.0
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Install the required versions of packages using the requirements file. Requirements file location: /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/validated_linux_requirements.txt. Alternatively, use remote target to avoid dependency management. \nPackage name/Required version/Installed version\nazureml-core/pyopenssl<20.0.0/pyOpenSSL 20.0.0",
        "target": "azureml-core",
        "inner_error": {
            "code": "NotSupported",
     

In [18]:
displayHTML("<a href={} target='_blank'>Azure Portal: {}</a>".format(local_run.get_portal_url(), local_run.id))

NameError: name 'displayHTML' is not defined

In [ ]:
# Retrieve and save your best automl model.
best_run, fitted_model = local_run.get_output()


### YOUR CODE HERE ###